# 从 StuffDocumentsChain 迁移

[StuffDocumentsChain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.combine_documents.stuff.StuffDocumentsChain.html) 通过将文档串联到单个上下文窗口中来组合文档。它是用于问答、摘要和其他目的组合文档的一种直接有效的策略。

[create_stuff_documents_chain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html) 是推荐的替代方案。它的功能与 `StuffDocumentsChain` 相同，并对流和批处理功能提供了更好的支持。由于它是 [LCEL 原始组件](/docs/concepts/lcel) 的简单组合，因此也更容易扩展并集成到其他 LangChain 应用程序中。

下面我们将通过一个简单的示例来详细介绍 `StuffDocumentsChain` 和 `create_stuff_documents_chain`。

首先，让我们加载一个聊天模型：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [1]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

## 示例

让我们通过一个分析文档集的示例。我们首先生成一些简单的文档以作说明：

In [2]:
from langchain_core.documents import Document

documents = [
    Document(page_content="Apples are red", metadata={"title": "apple_book"}),
    Document(page_content="Blueberries are blue", metadata={"title": "blueberry_book"}),
    Document(page_content="Bananas are yelow", metadata={"title": "banana_book"}),
]

### 旧版
<details open>

下面我们展示使用 `StuffDocumentsChain` 的实现。我们为摘要任务定义了提示模板，并为此实例化一个 [LLMChain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.llm.LLMChain.html) 对象。我们定义了文档如何格式化到提示中，并确保各种提示中的键保持一致。

In [15]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

# This controls how each document will be formatted. Specifically,
# it will be passed to `format_document` - see that function for more
# details.
document_prompt = PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)
document_variable_name = "context"
# The prompt here should take as an input variable the
# `document_variable_name`
prompt = ChatPromptTemplate.from_template("Summarize this content: {context}")

llm_chain = LLMChain(llm=llm, prompt=prompt)
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)

现在我们可以调用我们的链：

In [19]:
result = chain.invoke(documents)
result["output_text"]

'This content describes the colors of different fruits: apples are red, blueberries are blue, and bananas are yellow.'

In [20]:
for chunk in chain.stream(documents):
    print(chunk)

{'input_documents': [Document(metadata={'title': 'apple_book'}, page_content='Apples are red'), Document(metadata={'title': 'blueberry_book'}, page_content='Blueberries are blue'), Document(metadata={'title': 'banana_book'}, page_content='Bananas are yelow')], 'output_text': 'This content describes the colors of different fruits: apples are red, blueberries are blue, and bananas are yellow.'}


</details>

### LCEL

<details open>

下面我们展示使用 `create_stuff_documents_chain` 的实现：

In [21]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Summarize this content: {context}")
chain = create_stuff_documents_chain(llm, prompt)

调用链条，我们会获得与之前类似的结果：

In [24]:
result = chain.invoke({"context": documents})
result

'This content describes the colors of different fruits: apples are red, blueberries are blue, and bananas are yellow.'

请注意，此实现支持输出 token 的流式传输：

In [26]:
for chunk in chain.stream({"context": documents}):
    print(chunk, end=" | ")

 | This |  content |  describes |  the |  colors |  of |  different |  fruits | : |  apples |  are |  red | , |  blue | berries |  are |  blue | , |  and |  bananas |  are |  yellow | . |  | 

</details>

## 后续步骤

请查看 [LCEL 概念文档](/docs/concepts/lcel) 以获取更多背景信息。

请参阅这些[操作指南](/docs/how_to/#qa-with-rag)，了解更多关于使用 RAG 进行问答任务的内容。

请参阅[本教程](/docs/tutorials/summarization/)，了解更多基于 LLM 的摘要策略。